In [ ]:
"""
1. dataset of sentences to overall sentiment
2. create a model from dataset of word to sentiment
2.5 gramatically break down sentence instead of word by word
3. use model for our own sentence input to output overall sentiment of sentence
3.5 if a word is not known --> do sentiment analysis on its dictionary definition or neutralize
"""

In [72]:
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
import re
from sklearn.model_selection import train_test_split

lm = WordNetLemmatizer()

In [43]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lindseychin/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [38]:
syns = lm.lemmatize("is")
syns

'is'

In [60]:
col_names = ["sentiment", "ids", "date", "flag", "user", "text"]
df = pd.read_csv(
    'training.1600000.processed.noemoticon.csv',
    header=None, encoding="ISO-8859-1", names=col_names)
df2 = df.head(10000)

In [92]:
model = {}
STOP_WORDS_WO_NEG = stopwords.words('english')
STOP_WORDS_WO_NEG.remove('not')
STOP_WORDS_WO_NEG.remove('no')

URL_PATTERN = r'((https://[^ ]*|(https://)[^ ]*|( www\.)[^ ]*))'
USER_PATTERN = '@[^\s]+'
PUNCTUATIONS = ['!', '?']
NOT_PATTERN = r'\bnot \b'

for sentiment, tweet in zip(X_test['sentiment'], X_test['text']):
    tweet = tweet.lower()
    tweet = re.sub(URL_PATTERN, '', tweet)
    tweet = re.sub(USER_PATTERN, '', tweet)
    for p in PUNCTUATIONS:
        tweet = tweet.replace(p, '')
    for sw in STOP_WORDS_WO_NEG:
        tweet = re.sub(r'\b{0}\b'.format(sw), '', tweet)
    tweet = re.sub(NOT_PATTERN, 'not_', tweet)
    
    for word in tweet.split():
        word = lm.lemmatize(word)
        if word in model:
            count, avg = model[word]
            model[word] = (count+1, (count*avg+sentiment) / (count+1))
        else:
            model[word] = (1, sentiment)
            
model
        

{'want': (2323, 1.1967283684890255),
 'compete': (3, 0.0),
 'hard': (407, 1.3759213759213749),
 'competition': (25, 2.72),
 'rally.': (2, 2.0),
 'feel': (1729, 1.061885482938112),
 'power': (94, 1.702127659574468),
 'coming': (494, 1.9919028340080975),
 'engine': (12, 0.6666666666666666),
 'seems': (287, 1.4494773519163766),
 'stuck': (262, 0.5038167938931295),
 'ground': (24, 1.6666666666666667),
 'amarillo.': (1, 0),
 'put': (434, 2.036866359447005),
 'stop': (386, 1.5544041450777208),
 'flight': (136, 1.4411764705882357),
 'leaving': (270, 0.933333333333334),
 'denver.': (3, 0.0),
 'said': (419, 1.928400954653938),
 'update': (294, 1.673469387755102),
 'hour': (848, 1.415094339622642),
 'f': (29, 2.206896551724138),
 'pinking': (1, 0),
 'shear': (2, 0.0),
 'rarararrrarararr...babyproofing': (1, 0),
 'cutting': (27, 1.4814814814814814),
 'stuff': (378, 2.0846560846560847),
 'make': (1675, 1.9343283582089557),
 'stick': (64, 1.625),
 'random': (85, 2.0235294117647045),
 'place': (307,

In [70]:
df2.loc[df2['sentiment'] == 4]

,sentiment,ids,date,flag,user,text


In [73]:
X_train, X_test, y_train, y_test = train_test_split(df, df['sentiment'], test_size = 0.05, random_state=0)